In [1]:
!pip install pyspark

Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
    Running setup.py install for pyspark: started
    Running setup.py install for pyspark: finished with status 'done'


You should consider upgrading via the 'C:\Users\Fili\Desktop\Challenges\django-angular-app\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import pyspark

In [6]:
import pandas as pd
pd.read_csv('data.csv')


,name,age
0,fran,12
1,ana,34
2,vale,23


In [7]:
from pyspark.sql import SparkSession

In [22]:
spark=SparkSession.builder.appName('Dataframe').getOrCreate()

In [9]:
spark

In [87]:
# ways of reading data
rawData=spark.read.csv('data.csv')
# df_pyspark=spark.read.option('header', 'true').csv('data.csv', inferSchema=True)
df_pyspark=spark.read.csv('data.csv',header=True, inferSchema=True)
df_pyspark.show()


+----+----+---+-----+
|name| age|exp|  sal|
+----+----+---+-----+
|fran|  12|  1|30000|
| ana|null|  4| 2000|
|vale|  23| 20|50000|
|serg|  45|  2| null|
+----+----+---+-----+



In [30]:
# to see the type
# type(rawData)
type(df_pyspark)
# to see table or schema structure
# df_pyspark.printSchema()

pyspark.sql.dataframe.DataFrame

In [31]:
# see dara as a List
df_pyspark.head(3)

[Row(name='fran', age=12, exp=1),
 Row(name='ana', age=34, exp=4),
 Row(name='vale', age=23, exp=20)]

In [34]:
df_pyspark.select('name', 'exp').show()
# df_pyspark.select(['name', 'exp']).show()

+----+---+
|name|exp|
+----+---+
|fran|  1|
| ana|  4|
|vale| 20|
+----+---+



In [35]:
df_pyspark.describe()

DataFrame[summary: string, name: string, age: string, exp: string]

In [45]:
# Adding columns
df_pyspark=df_pyspark.withColumn('exp after 2yrs', df_pyspark['exp']+2)

In [46]:
df_pyspark.show()

+----+---+---+--------------+
|name|age|exp|exp after 2yrs|
+----+---+---+--------------+
|fran| 12|  1|             3|
| ana| 34|  4|             6|
|vale| 23| 20|            22|
+----+---+---+--------------+



In [47]:
df_pyspark=df_pyspark.drop('exp after 2yrs')
df_pyspark.show()

+----+---+---+
|name|age|exp|
+----+---+---+
|fran| 12|  1|
| ana| 34|  4|
|vale| 23| 20|
+----+---+---+



In [48]:
### Rename Cols
df_pyspark.withColumnRenamed('name', 'firstname')
df_pyspark.show()

+----+---+---+
|name|age|exp|
+----+---+---+
|fran| 12|  1|
| ana| 34|  4|
|vale| 23| 20|
+----+---+---+



In [54]:
# drop rows with Null values
# df_pyspark.na.drop().show()
df_pyspark=df_pyspark.na.drop()
#drop the ones with at least 2 Nulls
# df_pyspark=df_pyspark.na.drop(how='any',thresh=2)
df_pyspark.show()

+----+---+---+-----+
|name|age|exp|  sal|
+----+---+---+-----+
|fran| 12|  1|30000|
|vale| 23| 20|50000|
+----+---+---+-----+



In [58]:
#subset property of Drop - it removes rows from specific cols with nulls
# df_pyspark.show()
df_pyspark.na.drop(how='any',subset=['age']).show()


+----+---+---+-----+
|name|age|exp|  sal|
+----+---+---+-----+
|fran| 12|  1|30000|
|vale| 23| 20|50000|
|serg| 45|  2| null|
+----+---+---+-----+



In [79]:
### filling nulls or missing vals
df_pyspark.na.fill('Missing').show() 

+----+----+---+-----+
|name| age|exp|  sal|
+----+----+---+-----+
|fran|  12|  1|30000|
| ana|null|  4| 2000|
|vale|  23| 20|50000|
|serg|  45|  2| null|
+----+----+---+-----+



In [74]:
from pyspark.ml.feature import Imputer
imputer = Imputer(
    inputCols=["age","sal"],
    outputCols=["{}_imputed".format(c) for c in ["age","sal"]]
).setStrategy("mean")

In [75]:
# add imputations to DF
imputer.fit(df_pyspark).transform(df_pyspark).show()

+----+----+---+-----+-----------+-----------+
|name| age|exp|  sal|age_imputed|sal_imputed|
+----+----+---+-----+-----------+-----------+
|fran|  12|  1|30000|         12|      30000|
| ana|null|  4| 2000|         26|       2000|
|vale|  23| 20|50000|         23|      50000|
|serg|  45|  2| null|         45|      27333|
+----+----+---+-----+-----------+-----------+



In [92]:
### Operations
df_pyspark.filter('sal>2000').select(['name','age']).show()


+----+---+
|name|age|
+----+---+
|fran| 12|
|vale| 23|
+----+---+



In [98]:
df_pyspark.filter(df_pyspark['sal']>2000).show()

+----+---+---+-----+
|name|age|exp|  sal|
+----+---+---+-----+
|fran| 12|  1|30000|
|vale| 23| 20|50000|
+----+---+---+-----+



In [101]:
### using AND
df_pyspark.filter((df_pyspark['sal']>2000) & (df_pyspark['age']>12)).show()

+----+---+---+-----+
|name|age|exp|  sal|
+----+---+---+-----+
|vale| 23| 20|50000|
+----+---+---+-----+



In [100]:
### using OR
df_pyspark.filter((df_pyspark['sal']>2000) | (df_pyspark['age']>12)).show()

+----+---+---+-----+
|name|age|exp|  sal|
+----+---+---+-----+
|fran| 12|  1|30000|
|vale| 23| 20|50000|
|serg| 45|  2| null|
+----+---+---+-----+



In [125]:
data2=spark.read.csv('data2.csv',header=True, inferSchema=True,sep=';')

### using GROUP BY - find the cost of each department
data2.groupBy('dep').sum().show()


+--------+--------+--------+--------+
|     dep|sum(age)|sum(sal)|sum(exp)|
+--------+--------+--------+--------+
|frontend|      22|    1500|       1|
|big data|     156|   18000|      25|
|     iot|      35|    3000|       2|
|    data|      44|    2500|       2|
| backend|      36|    1500|       2|
+--------+--------+--------+--------+



In [126]:
data2.groupBy('dep').count().show()

+--------+-----+
|     dep|count|
+--------+-----+
|frontend|    1|
|big data|    3|
|     iot|    1|
|    data|    2|
| backend|    1|
+--------+-----+



In [127]:
data2.agg({'sal':'sum'}).show()

+--------+
|sum(sal)|
+--------+
|   26500|
+--------+



In [128]:
### getting Max Salaries per Department
data2.groupBy('dep').max().show()

+--------+--------+--------+--------+
|     dep|max(age)|max(sal)|max(exp)|
+--------+--------+--------+--------+
|frontend|      22|    1500|       1|
|big data|      66|   10000|      20|
|     iot|      35|    3000|       2|
|    data|      23|    1300|       1|
| backend|      36|    1500|       2|
+--------+--------+--------+--------+



In [129]:
training = data2
training.show()

+------+--------+---+-----+---+
|  name|     dep|age|  sal|exp|
+------+--------+---+-----+---+
|  fran|    data| 23| 1300|  1|
|   ana|big data| 45| 4000|  2|
|sergio|frontend| 22| 1500|  1|
| maria|     iot| 35| 3000|  2|
| lucia|big data| 66|10000| 20|
| pedro|big data| 45| 4000|  3|
|  neia|    data| 21| 1200|  1|
|chiqui| backend| 36| 1500|  2|
+------+--------+---+-----+---+



In [130]:
from pyspark.ml.feature import VectorAssembler
### mix cols to create an Indep Feature called Result
feature=VectorAssembler(inputCols=['age','exp'],outputCol='result')


In [131]:
result=feature.transform(data2)

In [132]:
result.show()

+------+--------+---+-----+---+-----------+
|  name|     dep|age|  sal|exp|     result|
+------+--------+---+-----+---+-----------+
|  fran|    data| 23| 1300|  1| [23.0,1.0]|
|   ana|big data| 45| 4000|  2| [45.0,2.0]|
|sergio|frontend| 22| 1500|  1| [22.0,1.0]|
| maria|     iot| 35| 3000|  2| [35.0,2.0]|
| lucia|big data| 66|10000| 20|[66.0,20.0]|
| pedro|big data| 45| 4000|  3| [45.0,3.0]|
|  neia|    data| 21| 1200|  1| [21.0,1.0]|
|chiqui| backend| 36| 1500|  2| [36.0,2.0]|
+------+--------+---+-----+---+-----------+



In [134]:
### out = Sal in = result
finalized = result.select('result','exp')
finalized.show()

+-----------+---+
|     result|exp|
+-----------+---+
| [23.0,1.0]|  1|
| [45.0,2.0]|  2|
| [22.0,1.0]|  1|
| [35.0,2.0]|  2|
|[66.0,20.0]| 20|
| [45.0,3.0]|  3|
| [21.0,1.0]|  1|
| [36.0,2.0]|  2|
+-----------+---+

